In [1]:
from ROOT import gROOT, gDirectory, gPad, gSystem, gStyle
from ROOT import TCanvas, TFile, TProfile, TNtuple, TH1F, TH2F, THStack, TLegend
from ROOT import TGaxis, TCut
from ROOT import kWhite, kBlack, kGray, kRed, kGreen, kBlue, kYellow, kMagenta, kCyan, kOrange, kSpring, kTeal, kAzure, kViolet, kPink
%jsroot on

Welcome to JupyROOT 6.18/00


In [2]:
def makePretty(cosAll, color, reconumber):
    cosAll.Scale(1./reconumber)
    cosAll.SetLineColor(color)
    cosAll.SetLineWidth(2)

# Main code starts here

In [3]:
gStyle.SetOptFit(0)
gStyle.SetOptStat(0)  
gStyle.SetOptTitle(1)
gStyle.SetTitleBorderSize(0)
gStyle.SetTitleStyle(0)
gStyle.SetMarkerSize(1)
gStyle.SetTitleX(0.2) 
gStyle.SetTitleY(0.9)

In [4]:
# before
#file = TFile.Open("/hsm/ilc/users/yokugawa/preset_N_run/l5/semiLep/eLpR/electron_muon/TrashRecoProcessor_out/after_vtx_recovery/IsoLepTagged.eL.pR_electron_muon_TRP_after.root")
#file = TFile.Open("/group/ilc/users/yokugawa/TTbar/l5/eLpR/semiLep/electron_muon/TrashRecoProcessor_out/after_vtx_recovery/IsoLepTagged.eLpR_electron_muon_TRP_after_121720.root")
file = TFile.Open("/group/ilc/users/yokugawa/TTbar/l5/eLpR/semiLep/electron_muon/TrashRecoProcessor_out/before_vtx_recovery/IsoLepTagged.eLpR_electron_muon_TRP_before_121720.root")

## Missed Vertices
### Create Histograms

In [5]:
bin_e = 50
max_e = 1

In [6]:
c1 = TCanvas("c1", "Data-MC",0,0,500,500)

cosReco = TH1F("cosReco", "E(Ntracks)", bin_e,.0,max_e)
cosOne = TH1F("cosOne", "E(Ntracks)", bin_e,.0,max_e)
cosOtherLow = TH1F("cosOtherLow", "tracks", bin_e,.0,max_e)
cosOtherHigh = TH1F("cosOtherHigh", "tracks", bin_e,.0,max_e)
cosOtherLowDist = TH1F("cosOtherLowDist", "tracks", bin_e,.0,max_e)

In [7]:
normaltree = file.Get("MissedVertex")

In [8]:
cosOne.SetLineColor(kBlue)
cosOne.SetLineWidth(3)
cosOtherHigh.SetLineColor(kRed)
cosOtherHigh.SetLineWidth(3)
cosOtherLowDist.SetLineColor(kGreen)
cosOtherLowDist.SetLineWidth(3)
cosOtherLowDist.SetTitle("|cos#theta_{vertex}|")

In [9]:
Stats = file.Get("Stats")
vtxnumber = Stats.GetEntries()

In [10]:
TaggedVertices = file.Get("TaggedVertices")

### Fill Histograms

In [11]:
reconumber = TaggedVertices.Draw("abs(costhetaOfParticles) >> cosReco","momentumOfParticles > 0")
normaltree.Draw("abs(costhetaMissedVtx) >> cosOne","numberOfTracksMissedVtx < 2")
normaltree.Draw("abs(costhetaMissedVtx) >> cosOtherHigh","numberOfTracksMissedVtx > 1 && averagepMissedVtx > 4.0 && averagesMissedVtx > .1")
normaltree.Draw("abs(costhetaMissedVtx) >> cosOtherLowDist","numberOfTracksMissedVtx > 1 && ( averagepMissedVtx < 4.0 || averagesMissedVtx < .5)")

522623

In [12]:
stack = THStack("stack",";|cos#theta_{vtx}|; Entries (normalised to N_{rec})")

cosOtherLowDist.Scale(1./vtxnumber);
cosOne.Scale(1./vtxnumber);
cosOtherHigh.Scale(1./vtxnumber);
stack.Add(cosOtherLowDist);
stack.Add(cosOne);
stack.Add(cosOtherHigh);
gPad.SetLeftMargin(0.18);
gPad.SetRightMargin(0.05);

stack.Draw("histo");
stack.GetYaxis().SetTitleOffset(1.7);

In [13]:
legendMean = TLegend(0.2,0.7,0.63,0.85,"","brNDC")

legendMean.SetFillColor(kWhite)
legendMean.SetBorderSize(0)
legendMean.AddEntry(cosOtherHigh,"Other vertices","f")
legendMean.AddEntry(cosOne,"0-1 prong decay","f")
legendMean.AddEntry(cosOtherLowDist,"Low momentum or low offset","f")
legendMean.Draw()

stack.GetXaxis().SetTitle("|cos#theta|")
gPad.Modified()


In [14]:
c1.Draw()

## Missed Tracks

In [15]:
c2 = TCanvas("c2", "Data-MC",0,0,500,500)
tree = file.Get("MissedTracks")

### Create Histograms

In [16]:
cosLowtr      = TH1F("cosLowtr", "tracks", bin_e,.0,max_e)
cosGoodTracks = TH1F("cosGoodTracks", "tracks", bin_e,.0,max_e)
cosNoPFO      = TH1F("cosNoPFO", "tracks", bin_e,.0,max_e)
cosNoVTX      = TH1F("cosNoVTX", "tracks", bin_e,.0,max_e)
cosNoFTD      = TH1F("cosNoFTD", "tracks", bin_e,.0,max_e)
cosNoTrack    = TH1F("cosNoTrack", "tracks", bin_e,.0,max_e)
cosBtag       = TH1F("cosBtag", "tracks", bin_e,.0,max_e)
cosAll        = TH1F("cosAll", "tracks", bin_e,.0,max_e)

### Fill

In [17]:
cosP = TCut("MCTopcostheta > 0")
cosN = TCut("MCTopcostheta < 0 && MCTopcostheta > -2")

CutGoodTracks = TCut("isrecoMissed > 0 && (vtxHitsMissed > 0 || ftdHitsMissed > 0 ) && momentumMissed > 1.0 && offsetMissed > 0.015")
CutLowtr = TCut("isrecoMissed > 0 && (vtxHitsMissed > 0 || ftdHitsMissed > 0 ) && (deviationMissed < 25*sqrt(vertexAngleMissed)+2 || vertexAngleMissed > 0.08)")
CutNoPFO = TCut("isrecoMissed == 0 && hastrackMissed == 1")
CutNoTrack = TCut("hastrackMissed == 0")
CutNoVTX = TCut("ftdHitsMissed == 0 && vtxHitsMissed == 0 && isrecoMissed == 1")
CutBtag = TCut("(vtxHitsMissed > 0 || ftdHitsMissed > 0) && isrecoMissed == 1 && (btagMissed < 0.8)")

In [18]:
tree.Draw("abs(costhetaMissed) >> cosAll",                       cosN)
tree.Draw("abs(costhetaMissed) >> cosGoodTracks",CutGoodTracks + cosN)
tree.Draw("abs(costhetaMissed) >> cosLowtr",CutLowtr           + cosN)
tree.Draw("abs(costhetaMissed) >> cosNoPFO",CutNoPFO           + cosN)
tree.Draw("abs(costhetaMissed) >> cosNoTrack",CutNoTrack       + cosN)
tree.Draw("abs(costhetaMissed) >> cosNoVTX",CutNoVTX           + cosN)
tree.Draw("abs(costhetaMissed) >> cosBtag",CutBtag             + cosN)

119837

In [19]:
print("Vertex total: ", cosAll.Integral())
print("no track:     ", cosNoTrack.Integral())
print("no vxd hits:  ", cosNoVTX.Integral())
print("no PFO:       ", cosNoPFO.Integral())
print("recoverable:  ",  cosGoodTracks.Integral())

Vertex total:  559347.0
no track:      53478.0
no vxd hits:   63138.0
no PFO:        34606.0
recoverable:   327643.0


In [20]:
makePretty(cosAll, kRed, reconumber)
makePretty(cosNoVTX, kBlue, reconumber)
makePretty(cosNoFTD, kYellow, reconumber)
makePretty(cosNoTrack, kBlack, reconumber)
makePretty(cosLowtr, kMagenta, reconumber)
makePretty(cosNoPFO, kGreen, reconumber)

### Stack

In [21]:
stack2 = THStack("stack2","|cos#theta| Entries (normalised to N_{rec})")

stack2.Add(cosNoTrack)
stack2.Add(cosNoVTX)
stack2.Add(cosNoPFO)
cosLowtr.SetFillStyle(3005)
stack2.Add(cosLowtr)
cosAll.SetMinimum(0)
stack2.SetMinimum(0)
stack2.SetMaximum(cosAll.GetMaximum())
gPad.SetLeftMargin(0.18)
gPad.SetRightMargin(0.05)
cosAll.Draw("histo")
stack2.Draw("histosame")
stack2.GetYaxis().SetTitleOffset(1.7)
stack2.GetYaxis().SetTitle("|cos#theta|")

In [22]:
legendMean2 = TLegend(0.2,0.6,0.63,0.85,"","brNDC");
legendMean2.SetFillColor(kWhite);
legendMean2.SetBorderSize(0);
legendMean2.AddEntry(cosAll,"Other reasons","f");
legendMean2.AddEntry(cosLowtr,"Low offset significance","f");
legendMean2.AddEntry(cosNoPFO,"Nonreconstructed PFO","f");
legendMean2.AddEntry(cosNoVTX,"No hits in VXD&FTD","f");
legendMean2.AddEntry(cosNoTrack,"No tracking information","f");
legendMean2.Draw();

In [23]:
c2.Draw()